# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [1]:
import os
import keras
import itertools

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 50
BATCH_SIZE = 256
OPTIMIZER = [keras.optimizers.SGD, keras.optimizers.RMSprop, keras.optimizers.Adagrad, keras.optimizers.Adam]

In [ ]:
results = {}
for lr, opti in itertools.product(LEARNING_RATE, OPTIMIZER):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with LR = %.6f, Optimizer = %s" % (lr, str(opti)))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    
    optimizer = opti(lr=lr)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-lr-%s-optimizer-%s" % (str(lr), str(opti))
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

Experiment with LR = 0.100000, Optimizer = <class 'keras.optimizers.SGD'>
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000

50000/50000 [==============================] - 6s 111us/step - loss: 0.7810 - accuracy: 0.7244 - val_loss: 1.6321 - val_accuracy: 0.5109
Epoch 49/50
50000/50000 [==============================] - 5s 110us/step - loss: 0.7681 - accuracy: 0.7264 - val_loss: 1.7068 - val_accuracy: 0.4995
Epoch 50/50
50000/50000 [==============================] - 6s 111us/step - loss: 0.7592 - accuracy: 0.7316 - val_loss: 1.8193 - val_accuracy: 0.4642
Experiment with LR = 0.100000, Optimizer = <class 'keras.optimizers.RMSprop'>
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
________

50000/50000 [==============================] - 7s 139us/step - loss: 2.3076 - accuracy: 0.1004 - val_loss: 2.3095 - val_accuracy: 0.1000
Epoch 46/50
50000/50000 [==============================] - 7s 139us/step - loss: 2.3079 - accuracy: 0.1001 - val_loss: 2.3095 - val_accuracy: 0.1000
Epoch 47/50
50000/50000 [==============================] - 8s 153us/step - loss: 2.3075 - accuracy: 0.1009 - val_loss: 2.3098 - val_accuracy: 0.1000
Epoch 48/50
50000/50000 [==============================] - 7s 144us/step - loss: 2.3078 - accuracy: 0.1021 - val_loss: 2.3117 - val_accuracy: 0.1000
Epoch 49/50
50000/50000 [==============================] - 7s 146us/step - loss: 2.3079 - accuracy: 0.1006 - val_loss: 2.3146 - val_accuracy: 0.1000
Epoch 50/50
50000/50000 [==============================] - 7s 142us/step - loss: 2.3079 - accuracy: 0.0985 - val_loss: 2.3155 - val_accuracy: 0.1000
Experiment with LR = 0.100000, Optimizer = <class 'keras.optimizers.Adagrad'>
Model: "model_1"
_______________________

50000/50000 [==============================] - 7s 132us/step - loss: 1.5210 - accuracy: 0.4544 - val_loss: 1.5940 - val_accuracy: 0.4269
Epoch 43/50
50000/50000 [==============================] - 7s 130us/step - loss: 1.5178 - accuracy: 0.4545 - val_loss: 1.5985 - val_accuracy: 0.4269
Epoch 44/50
50000/50000 [==============================] - 7s 132us/step - loss: 1.5124 - accuracy: 0.4542 - val_loss: 1.6022 - val_accuracy: 0.4196
Epoch 45/50
50000/50000 [==============================] - 7s 131us/step - loss: 1.5071 - accuracy: 0.4583 - val_loss: 1.5884 - val_accuracy: 0.4313
Epoch 46/50
50000/50000 [==============================] - 7s 132us/step - loss: 1.5043 - accuracy: 0.4579 - val_loss: 1.6556 - val_accuracy: 0.4076
Epoch 47/50
50000/50000 [==============================] - 7s 133us/step - loss: 1.4990 - accuracy: 0.4606 - val_loss: 1.6290 - val_accuracy: 0.4144
Epoch 48/50
50000/50000 [==============================] - 7s 131us/step - loss: 1.4940 - accuracy: 0.4625 - val_loss:

50000/50000 [==============================] - 9s 170us/step - loss: 2.3063 - accuracy: 0.1002 - val_loss: 2.3047 - val_accuracy: 0.1000
Epoch 40/50
50000/50000 [==============================] - 8s 157us/step - loss: 2.3065 - accuracy: 0.1001 - val_loss: 2.3059 - val_accuracy: 0.1000
Epoch 41/50
50000/50000 [==============================] - 8s 156us/step - loss: 2.3080 - accuracy: 0.0985 - val_loss: 2.3049 - val_accuracy: 0.1000
Epoch 42/50
50000/50000 [==============================] - 9s 182us/step - loss: 2.3068 - accuracy: 0.1005 - val_loss: 2.3046 - val_accuracy: 0.1000
Epoch 43/50
50000/50000 [==============================] - 9s 184us/step - loss: 2.3082 - accuracy: 0.0994 - val_loss: 2.3121 - val_accuracy: 0.1000
Epoch 44/50
50000/50000 [==============================] - 9s 176us/step - loss: 2.3068 - accuracy: 0.1007 - val_loss: 2.3084 - val_accuracy: 0.1000
Epoch 45/50
50000/50000 [==============================] - 8s 162us/step - loss: 2.3070 - accuracy: 0.0980 - val_loss:

...

50000/50000 [==============================] - 7s 146us/step - loss: 1.2294 - accuracy: 0.5622 - val_loss: 1.3481 - val_accuracy: 0.5223
Epoch 16/50
50000/50000 [==============================] - 7s 146us/step - loss: 1.2036 - accuracy: 0.5686 - val_loss: 1.3568 - val_accuracy: 0.5187
Epoch 17/50
50000/50000 [==============================] - 7s 147us/step - loss: 1.1896 - accuracy: 0.5753 - val_loss: 1.3990 - val_accuracy: 0.5068
Epoch 18/50
50000/50000 [==============================] - 7s 144us/step - loss: 1.1579 - accuracy: 0.5866 - val_loss: 1.3756 - val_accuracy: 0.5222
Epoch 19/50
50000/50000 [==============================] - 7s 147us/step - loss: 1.1379 - accuracy: 0.5940 - val_loss: 1.3599 - val_accuracy: 0.5274
Epoch 20/50
50000/50000 [==============================] - 8s 162us/step - loss: 1.1219 - accuracy: 0.6017 - val_loss: 1.3515 - val_accuracy: 0.5243
Epoch 21/50
50000/50000 [==============================] - 7s 148us/step - loss: 1.0949 - accuracy: 0.6086 - val_loss:

50000/50000 [==============================] - 6s 116us/step - loss: 2.2059 - accuracy: 0.2112 - val_loss: 2.2031 - val_accuracy: 0.2194
Epoch 13/50
50000/50000 [==============================] - 6s 118us/step - loss: 2.1995 - accuracy: 0.2147 - val_loss: 2.1969 - val_accuracy: 0.2233
Epoch 14/50
50000/50000 [==============================] - 6s 120us/step - loss: 2.1933 - accuracy: 0.2198 - val_loss: 2.1909 - val_accuracy: 0.2273
Epoch 15/50
50000/50000 [==============================] - 6s 118us/step - loss: 2.1875 - accuracy: 0.2237 - val_loss: 2.1852 - val_accuracy: 0.2311
Epoch 16/50
50000/50000 [==============================] - 6s 116us/step - loss: 2.1819 - accuracy: 0.2269 - val_loss: 2.1797 - val_accuracy: 0.2334
Epoch 17/50
50000/50000 [==============================] - 6s 118us/step - loss: 2.1764 - accuracy: 0.2302 - val_loss: 2.1744 - val_accuracy: 0.2354
Epoch 18/50
50000/50000 [==============================] - 6s 119us/step - loss: 2.1712 - accuracy: 0.2340 - val_loss:

50000/50000 [==============================] - 7s 146us/step - loss: 1.4858 - accuracy: 0.4763 - val_loss: 1.4935 - val_accuracy: 0.4695
Epoch 10/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.4653 - accuracy: 0.4842 - val_loss: 1.7135 - val_accuracy: 0.4029
Epoch 11/50
50000/50000 [==============================] - 7s 142us/step - loss: 1.4416 - accuracy: 0.4909 - val_loss: 1.4883 - val_accuracy: 0.4706
Epoch 12/50
50000/50000 [==============================] - 7s 136us/step - loss: 1.4205 - accuracy: 0.4997 - val_loss: 1.5832 - val_accuracy: 0.4419
Epoch 13/50
50000/50000 [==============================] - 7s 143us/step - loss: 1.3990 - accuracy: 0.5094 - val_loss: 1.4651 - val_accuracy: 0.4765
Epoch 14/50
50000/50000 [==============================] - 7s 141us/step - loss: 1.3826 - accuracy: 0.5139 - val_loss: 1.4644 - val_accuracy: 0.4793
Epoch 15/50
50000/50000 [==============================] - 7s 140us/step - loss: 1.3641 - accuracy: 0.5182 - val_loss:

50000/50000 [==============================] - 7s 132us/step - loss: 1.8300 - accuracy: 0.3673 - val_loss: 1.8274 - val_accuracy: 0.3680
Epoch 7/50
50000/50000 [==============================] - 7s 132us/step - loss: 1.8170 - accuracy: 0.3716 - val_loss: 1.8137 - val_accuracy: 0.3751
Epoch 8/50
50000/50000 [==============================] - 7s 138us/step - loss: 1.8060 - accuracy: 0.3763 - val_loss: 1.8031 - val_accuracy: 0.3747
Epoch 9/50
50000/50000 [==============================] - 7s 139us/step - loss: 1.7963 - accuracy: 0.3806 - val_loss: 1.7942 - val_accuracy: 0.3821
Epoch 10/50
50000/50000 [==============================] - 7s 131us/step - loss: 1.7879 - accuracy: 0.3816 - val_loss: 1.7864 - val_accuracy: 0.3846
Epoch 11/50
50000/50000 [==============================] - 7s 140us/step - loss: 1.7796 - accuracy: 0.3846 - val_loss: 1.7780 - val_accuracy: 0.3884
Epoch 12/50
50000/50000 [==============================] - 7s 137us/step - loss: 1.7721 - accuracy: 0.3885 - val_loss: 1.

50000/50000 [==============================] - 7s 149us/step - loss: 1.6310 - accuracy: 0.4247 - val_loss: 1.6165 - val_accuracy: 0.4317
Epoch 4/50
50000/50000 [==============================] - 8s 152us/step - loss: 1.5817 - accuracy: 0.4425 - val_loss: 1.5556 - val_accuracy: 0.4535
Epoch 5/50
50000/50000 [==============================] - 8s 158us/step - loss: 1.5357 - accuracy: 0.4587 - val_loss: 1.5310 - val_accuracy: 0.4575
Epoch 6/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.5058 - accuracy: 0.4692 - val_loss: 1.5028 - val_accuracy: 0.4702
Epoch 7/50
50000/50000 [==============================] - 7s 150us/step - loss: 1.4719 - accuracy: 0.4820 - val_loss: 1.4832 - val_accuracy: 0.4735
Epoch 8/50
50000/50000 [==============================] - 8s 153us/step - loss: 1.4391 - accuracy: 0.4945 - val_loss: 1.4644 - val_accuracy: 0.4763
Epoch 9/50
50000/50000 [==============================] - 8s 156us/step - loss: 1.4157 - accuracy: 0.5018 - val_loss: 1.450

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
    
NUM_COLORS = len(results.keys())
cm = plt.get_cmap('gist_rainbow')
color_bar = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()